In [1]:
### Import NumPy and time
import numpy as np
import time

Randomized Hadamard sketch: The sketch matrix in this case can be represented as S = P*H*D where P ∈ R^m×n is for uniform sampling of m rows out of n rows, \
H ∈ R^n×n is the Hadamard matrix, and D ∈ R^n×n is a diagonal matrix with diagonal entries sampled randomly from the Rademacher distribution. \
Multiplication by D to obtain DA requires O(n*d) scalar multiplications. \
Hadamard transform can be implemented as a fast transform with complexity O(n*log(n)) per column, and a total complexity of O(n*d*log(n)) to sketch all d columns of DA.

In [2]:
### Matrix_P is for uniform sampling of m rows out of n rows
def matrix_P (m,n):
    I = np.eye(n)
    sample_indices = np.random.choice(n,m,replace = False)
    sample_indices.sort()
    P = I[sample_indices,:]
    return P

In [3]:
### Import scipy for Fasterest Consturction of Hadamard
import scipy.linalg
H = scipy.linalg.hadamard(2**10)

In [4]:
### Create a diagonal matrix with diagonal entries sampled randomly from the Rademacher distribution
vec = np.random.choice([-1,1],2**10)
D = np.diag(vec)

In [5]:
### Finally construct the sketch matrix
### S = matrix_P(mm_for_sampling,nn) @ H
### S = S @ D

In [6]:
def random_hadamard_sketch(S,A):
    Transformed = S @ A
    return Transformed
### A has nn rows

Now we need the padding method needed to be employed to accommodate date for arbitrary size.\
This involves adding zeros to the end of the data matrix X and the target vector y, ensuring that the resulting dimensions are powers of two.

In [7]:
def pad_to_power_of_two(X, y):
    """Pads the data matrix X and target vector y with zeros to the nearest power of two.
    Args:
    X: The data matrix.
    y: The target vector.
    Returns:
    The padded data matrix and target vector.
    """
    n_samples, n_features = X.shape

    # Find the next power of two
    next_power_of_two = 2**int(np.ceil(np.log2(n_samples)))

    # Pad the data matrix and target vector with zeros
    X_add = np.zeros((next_power_of_two - n_samples,n_features))
    X_padded = np.concatenate((X,X_add),axis=0)
    y_padded = np.pad(y, (0, next_power_of_two - n_samples), 'constant')
    return X_padded, y_padded

Our target: calculate the least square time and compared it with codes written in R language.

In [8]:
### Apply subsampled randomized hadamard transform and timer the time using the wind data.
import pandas as pd
data = pd.read_csv("wind.csv",header=None)
data = np.array(data)
Y = data[:,0]
X = data[:,1:len(data)]
start1 = time.time()
m = 2000
Xpad,Ypad = pad_to_power_of_two(X,Y)
Ypad = Ypad.reshape(-1,1)
st = time.time()
H = scipy.linalg.hadamard(len(Xpad))
ed = time.time()
time_hada = ed - st
print(time_hada*1000,"ms for Hadamard Matrix Forming.")
S = matrix_P(m,len(Xpad)) @ H
vec = np.random.choice([-1,1],len(Xpad))
D = np.diag(vec)
S = S @ D
data_n = np.concatenate((Xpad,Ypad),axis=1)
hadamard = random_hadamard_sketch(S,data_n)
Sx = hadamard[:,0:(hadamard.shape[1]-1)]
Sy = hadamard[:,(hadamard.shape[1]-1)]
solution = np.linalg.inv(Sx.T@Sx)@X.T@Y
### Sppose the test input = [1,2,...,14]
test = (np.arange(1,15)).reshape(-1,1)
### Y_expect = sum(solution)
Y_expect = sum(solution@test)
print(solution)
print(Y_expect)
print(hadamard)
end1 = time.time()
print("The time of execution of Randomized Hadamard Sketching is :",(end1-start1) * 10**3, "ms")

1206.7649364471436 ms for Hadamard Matrix Forming.
[ 0.00083472  0.0003015   0.00044266  0.00015157  0.00102233 -0.00245004
  0.00115447 -0.00133157  0.00029259 -0.0013722   0.00161695 -0.00013757
  0.0006867   0.00026506]
0.00889706744153719
[[  748.     996.    1049.14 ...  1853.02  1608.56  6431.  ]
 [  180.     958.      56.3  ...   128.66   -22.44 -1045.  ]
 [  372.    1638.     246.2  ...   384.04   237.12  3367.  ]
 ...
 [ -258.   -2158.    -331.34 ...  -481.36 -1041.9  -2799.  ]
 [  182.     278.     181.8  ...   182.44   354.36  2345.  ]
 [ 1210.    3028.    2037.3  ...  1693.14  2638.18 10709.  ]]
The time of execution of Randomized Hadamard Sketching is : 6387.497186660767 ms


Now Let Us Implement the algorithm written by Pro. Zhang in R

In [9]:
import math
def padding(X, y):
    m = X.shape[0]
    if (math.ceil(math.log2(m))>math.log2(m)):
        m_new = math.floor(math.log2(m))+1
        padX = np.concatenate([X, np.zeros((2**m_new - m, X.shape[1]), dtype=int)], axis=0)
        pady = np.append(y,np.zeros((2**m_new) - m,dtype=int))
    else:
        padX = X
        pady = y
    return padX, pady
def fht_1d(arr):
    """
    Performs Fast Hadamard Transform (FHT) on a 1D array.
    """
    n = arr.shape[0]
    assert (n & (n - 1)) == 0, "Length of the array must be a power of 2."

    result = arr.copy()
    h = 1
    while h < n:
        for i in range(0, n, h * 2):
            for j in range(h):
                x = result[i + j]
                y = result[i + j + h]
                result[i + j] = x + y
                result[i + j + h] = x - y
        h *= 2
    return result

def fht_2d_by_columns(matrix):
    """
    Performs Fast Hadamard Transform (FHT) on each column of a 2D array.
    
    Args:
        matrix (numpy.ndarray): 2D array where the number of rows is a power of 2.

    Returns:
        numpy.ndarray: Transformed 2D array with FHT applied column by column.
    """
    n_rows = matrix.shape[0]
    assert (n_rows & (n_rows - 1)) == 0, "Number of rows must be a power of 2."

    transformed = np.zeros_like(matrix)
    for col in range(matrix.shape[1]):
        transformed[:, col] = fht_1d(matrix[:, col])
    return transformed
def Esticoef_SRHT(m, c, X, y, partial=0):
    p = X.shape[1]
    X1, y1 = padding(X, y)
    n1 = X1.shape[0]
    gamma = m / n1

    random_signs = np.random.choice([1, -1], size=n1, p=[0.5, 0.5])[:, None]
    combined_data = np.column_stack((X1, y1))
    signed_data = random_signs*combined_data
    result = fht_2d_by_columns(signed_data)
    result = result [np.where(np.random.binomial(1, gamma, size=n1) != 0)[0]]
    SXy = result / math.sqrt(m)
    print(SXy)
    if partial == 0:
        try:
            g = np.linalg.solve(SXy[:, :p], SXy[:, p])
        except np.linalg.LinAlgError:
            print("Matrix is singular or ill-conditioned. Trying alternative approach.")
            g = np.linalg.lstsq(SXy[:, :p], SXy[:, p], rcond=None)[0]
    else:
        SX = SXy[:, :p]

        M = np.linalg.inv(SX.T @ SX)

        g = M @ (X.T @ y)

        return g, np.dot(c, g), SXy
    

Now Let Us Input The experimental data and see the resulted time recording.

In [10]:
start_time = time.time()
m = 2000
c = np.arange(1,15,1)
print(Esticoef_SRHT(m,c,X,Y,partial=1))
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time*1000,"ms for Python Subsampled Randomized Hadamard Transform.")

[[  11.85116028   39.5784032    12.81043344 ...   21.90139141
    25.73624799   65.31554562]
 [  13.99778554   50.04320134   47.90104821 ...   37.37990117
    43.88596455  169.42687066]
 [  -3.17521653   13.05863699   14.2365976  ...    7.453709
    -1.02769684   19.34198801]
 ...
 [  16.81523119   37.83427018   14.96823904 ...   21.24130415
    19.2462843    91.34337688]
 [  -5.90321946  -35.24043133  -22.49797435 ...  -15.56661083
   -21.14560044 -158.02292397]
 [   3.84603692  -11.53811076  -13.4799122  ...   -8.20010849
    -7.3848381    38.70633669]]
(array([ 1.29619299,  0.65517024,  0.8355099 ,  0.38997195,  2.64122341,
       -3.01132862,  1.58930829, -2.97600558, -0.58913345, -2.76835059,
        2.70701356, -1.40421291,  0.9784537 ,  1.80212523]), 7.01874831139682, array([[  11.85116028,   39.5784032 ,   12.81043344, ...,   21.90139141,
          25.73624799,   65.31554562],
       [  13.99778554,   50.04320134,   47.90104821, ...,   37.37990117,
          43.88596455,  169.4